In [1]:
pip install git+git://github.com/titipata/pubmed_parser.git

  Cloning git://github.com/titipata/pubmed_parser.git to /private/var/folders/vl/b2t9x4j11s73llszkqpqlst80000gn/T/pip-req-build-dze9zegc
  Running command git clone -q git://github.com/titipata/pubmed_parser.git /private/var/folders/vl/b2t9x4j11s73llszkqpqlst80000gn/T/pip-req-build-dze9zegc
  Created wheel for pubmed-parser: filename=pubmed_parser-0.2.2-py3-none-any.whl size=18318 sha256=c7d9b12fdb6266a65933598673d7c46412ac353a6ab4e70b7da6487cdd5a481c
  Stored in directory: /private/var/folders/vl/b2t9x4j11s73llszkqpqlst80000gn/T/pip-ephem-wheel-cache-_s48lbby/wheels/d5/65/90/b0620ddce4e089c02c4468f2a0eb7e482ba7b2f321247857b8
Successfully built pubmed-parser
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pubmed_parser as pp
import pandas as pd
from gzip import decompress
import nltk
nltk.download("punkt")

SIZE = 2 # how many files are we bothering with
parsed_files = []
fnames = ["pubmed21n" + str(num).zfill(4) + ".xml" for num in range(1,1 + SIZE)]
#%% getting + processing files
ftp_url = "https://ftp.ncbi.nlm.nih.gov/pubmed/baseline/"
for fname in fnames:
    r = requests.get(ftp_url + fname + '.gz')
    print('yoinked')
    data = decompress(r.content)
    print('decompressed')
    medline_data = pd.DataFrame.from_dict(pp.parse_medline_xml(data))
    print('parsed')
    medline_data = medline_data[['abstract', 'pubdate']]
    parsed_files.append(medline_data)
    print('appended')

#%% create dataframe of parsed files
pubmed_df = pd.concat(parsed_files, ignore_index=True)
#%% isolate sentences
#pull abstracts <= 2015
abstracts = [pubmed_df['abstract'][i] for i in pubmed_df.index if int(pubmed_df['pubdate'][i]) <= 2015]
sentences = []
for abstract in abstracts:
    sentences += nltk.tokenize.sent_tokenize(abstract)

[nltk_data] Downloading package punkt to /Users/fufu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


yoinked
decompressed
parsed
appended
yoinked
decompressed
parsed
appended


In [3]:
drugs = []
genes = []

df_drug = pd.read_csv('./drugs/drugs.tsv', sep='\t')
df_genes = pd.read_csv('./genes/genes.tsv', sep='\t')

drugs.extend(df_drug["Name"])
genes.extend(df_genes["Name"])

In [16]:
#%% narrow sentences down to only those with drug-gene relationships
usable_sentences = []
set_drugs = set(drugs)
set_genes = set(genes)


for sentence in sentences:
    drug, gene = False, False
    token_sentence = nltk.tokenize.word_tokenize(sentence)
    for token in token_sentence:
        if token in set_drugs:
            drug = True
        elif token in set_gene:
            gene = True
        if drug and gene:
            usable_sentences.append(sentence)
            break



In [17]:
all_unique_biomedical_sentences = list(set(usable_sentences))

In [20]:
# Write to csv file 
import csv

iterable_data = [[sentence] for sentence in all_unique_biomedical_sentences]  
# data to be written row-wise in csv fil
"""
for sentence in all_unique_biomedical_sentences:
    iterable_data.append([sentence])
"""

# opening the csv file in 'w+' mode
file = open('biomedical sentences.csv', 'w+', newline ='')
  
# writing the data into the file
with file:    
    write = csv.writer(file)
    write.writerows(iterable_data)